In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My Drive

Mounted at /content/drive
/content/drive/My Drive


In [2]:
import random
from datetime import datetime

import numpy as np
import pandas as pd
import warnings
import tensorflow as tf

warnings.filterwarnings('ignore')
random.seed(0)
np.random.seed(0)

train_data = pd.read_csv('data/open data/train.csv',index_col='index').drop([379, 24598], axis=0)
test_data = pd.read_csv('data/open data/test_x.csv',index_col='index')

In [15]:
train_data.head()

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,QfA,QfE,QgA,QgE,QhA,QhE,QiA,QiE,QjA,QjE,QkA,QkE,QlA,QlE,QmA,QmE,QnA,QnE,QoA,QoE,QpA,QpE,QqA,QqE,QrA,QrE,QsA,QsE,QtA,QtE,age_group,education,engnat,familysize,gender,hand,married,race,religion,tp01,tp02,tp03,tp04,tp05,tp06,tp07,tp08,tp09,tp10,urban,voted,wf_01,wf_02,wf_03,wr_01,wr_02,wr_03,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,3.0,363,4.0,1370,5.0,997,1.0,1024,2.0,1577,5.0,539,2.0,586,4.0,1095,5.0,1142,4.0,1287,4.0,883,4.0,851,2.0,851,5.0,816,2.0,579,2.0,924,2.0,366,2.0,876,2.0,633,1.0,1115,30s,2,1,4,Female,1,3,White,Other,2,2,2,1,2,1,7,4,4,3,1,2,0,0,0,0,1,0,0,1,0,1,1,0,1,0,1,1
1,5.0,647,5.0,1313,3.0,3387,5.0,2969,1.0,4320,3.0,2190,1.0,826,1.0,4082,5.0,1867,3.0,1264,5.0,2943,4.0,3927,1.0,4329,5.0,1828,1.0,1214,5.0,2414,5.0,1356,1.0,3039,4.0,4304,1.0,1346,20s,4,2,3,Female,1,1,Asian,Hindu,1,1,0,0,1,2,3,4,0,4,3,2,0,0,0,0,1,0,1,1,0,1,1,0,1,0,1,1
2,4.0,1623,1.0,1480,1.0,1021,4.0,3374,5.0,1333,1.0,531,4.0,1167,1.0,1016,3.0,2653,2.0,1569,5.0,998,5.0,2547,2.0,918,4.0,2153,2.0,1304,1.0,1131,5.0,937,4.0,1327,1.0,1170,1.0,1409,30s,3,1,3,Male,1,2,White,Other,2,3,1,5,3,4,2,6,1,3,2,1,0,0,1,1,1,0,1,1,0,1,1,1,1,0,1,1
3,3.0,504,3.0,2311,4.0,992,3.0,3245,1.0,357,2.0,1519,4.0,159,3.0,2275,5.0,2809,4.0,5614,3.0,3219,4.0,1296,4.0,9046,4.0,1216,4.0,1169,4.0,23868,3.0,581,4.0,8830,4.0,2392,5.0,1312,20s,4,2,0,Female,1,1,Asian,Hindu,2,4,1,1,1,3,1,3,1,3,3,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,1
4,1.0,927,1.0,707,5.0,556,2.0,1062,1.0,1014,2.0,628,1.0,991,1.0,1259,5.0,1153,5.0,1388,5.0,740,5.0,1181,4.0,547,2.0,575,1.0,754,4.0,1140,5.0,323,5.0,1070,1.0,583,2.0,1889,20s,3,1,2,Male,1,2,White,Agnostic,1,1,1,6,0,2,0,6,2,6,1,1,0,1,0,1,1,0,1,1,1,1,1,0,1,1,1,1


In [25]:
drop_list = ['QaE', 'QbE', 'QcE', 'QdE', 'QeE',
             'QfE', 'QgE', 'QhE', 'QiE', 'QjE',
             'QkE', 'QlE', 'QmE', 'QnE', 'QoE',
             'QpE', 'QqE', 'QrE', 'QsE', 'QtE',
             'index', 'hand']
replace_dict = {'education': str, 'engnat': str, 'married': str, 'urban': str}
train_y = train_data['voted']
train_x = train_data.drop(drop_list + ['voted'], axis=1)
test_x = test_data.drop(drop_list, axis=1)
train_x = train_x.astype(replace_dict)
test_x = test_x.astype(replace_dict)
train_x = pd.get_dummies(train_x)
test_x = pd.get_dummies(test_x)
# train_y = 2 - train_y.to_numpy()
# train_x = train_x.to_numpy()
# test_x = test_x.to_numpy()

# train_x[:, :20] = (train_x[:, :20] - 3.) / 2.
# test_x[:, :20] = (test_x[:, :20] - 3.) / 2
# train_x[:, 20] = (train_x[:, 20] - 5.) / 5.
# test_x[:, 20] = (test_x[:, 20] - 5.) / 5.
# train_x[:, 21:31] = (train_x[:, 21:31] - 3.5) / 3.5
# test_x[:, 21:31] = (test_x[:, 21:31] - 3.5) / 3.5

In [26]:
train_x

,QaA,QbA,QcA,QdA,QeA,QfA,QgA,QhA,QiA,QjA,QkA,QlA,QmA,QnA,QoA,QpA,QqA,QrA,QsA,QtA,familysize,tp01,tp02,tp03,tp04,tp05,tp06,tp07,tp08,tp09,tp10,wf_01,wf_02,wf_03,wr_01,wr_02,wr_03,wr_04,wr_05,wr_06,...,age_group_40s,age_group_50s,age_group_60s,education_0,education_1,education_2,education_3,education_4,engnat_0,engnat_1,engnat_2,gender_Female,gender_Male,married_0,married_1,married_2,married_3,race_Arab,race_Asian,race_Black,race_Indigenous Australian,race_Native American,race_Other,race_White,religion_Agnostic,religion_Atheist,religion_Buddhist,religion_Christian_Catholic,religion_Christian_Mormon,religion_Christian_Other,religion_Christian_Protestant,religion_Hindu,religion_Jewish,religion_Muslim,religion_Other,religion_Sikh,urban_0,urban_1,urban_2,urban_3
0,3.0,4.0,5.0,1.0,2.0,5.0,2.0,4.0,5.0,4.0,4.0,4.0,2.0,5.0,2.0,2.0,2.0,2.0,2.0,1.0,4,2,2,2,1,2,1,7,4,4,3,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
1,5.0,5.0,3.0,5.0,1.0,3.0,1.0,1.0,5.0,3.0,5.0,4.0,1.0,5.0,1.0,5.0,5.0,1.0,4.0,1.0,3,1,1,0,0,1,2,3,4,0,4,0,0,0,0,1,0,1,1,0,...,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
2,4.0,1.0,1.0,4.0,5.0,1.0,4.0,1.0,3.0,2.0,5.0,5.0,2.0,4.0,2.0,1.0,5.0,4.0,1.0,1.0,3,2,3,1,5,3,4,2,6,1,3,0,0,1,1,1,0,1,1,0,...,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
3,3.0,3.0,4.0,3.0,1.0,2.0,4.0,3.0,5.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,3.0,4.0,4.0,5.0,0,2,4,1,1,1,3,1,3,1,3,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
4,1.0,1.0,5.0,2.0,1.0,2.0,1.0,1.0,5.0,5.0,5.0,5.0,4.0,2.0,1.0,4.0,5.0,5.0,1.0,2.0,2,1,1,1,6,0,2,0,6,2,6,0,1,0,1,1,0,1,1,1,...,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45527,2.0,5.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,4.0,5.0,4.0,1.0,2.0,4.0,2.0,1.0,5.0,4.0,1,1,2,6,1,2,6,3,0,6,4,0,0,1,1,1,0,1,1,0,...,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
45528,2.0,3.0,4.0,1.0,3.0,2.0,2.0,1.0,2.0,5.0,2.0,5.0,3.0,2.0,4.0,4.0,2.0,3.0,4.0,4.0,2,2,2,2,3,1,3,3,4,1,5,0,0,1,1,1,1,1,1,0,...,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
45529,4.0,1.0,1.0,4.0,5.0,4.0,5.0,1.0,5.0,1.0,5.0,5.0,1.0,4.0,1.0,1.0,5.0,5.0,3.0,1.0,1,4,4,2,4,0,1,0,2,1,6,0,0,0,1,1,0,1,1,0,...,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
45530,1.0,3.0,4.0,2.0,1.0,1.0,1.0,1.0,5.0,1.0,5.0,5.0,4.0,3.0,1.0,1.0,4.0,4.0,4.0,1.0,2,1,2,0,4,0,1,3,2,2,4,0,0,0,1,1,0,1,1,0,...,1,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [144]:
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from tqdm import tqdm

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

train_data = pd.read_csv('data/open data/train.csv').drop([379, 24598], axis=0)
test_data = pd.read_csv('data/open data/test_x.csv')
drop_list = ['QaE', 'QbE', 'QcE', 'QdE', 'QeE',
             'QfE', 'QgE', 'QhE', 'QiE', 'QjE',
             'QkE', 'QlE', 'QmE', 'QnE', 'QoE',
             'QpE', 'QqE', 'QrE', 'QsE', 'QtE',
             'index', 'hand']
replace_dict = {'education': str, 'engnat': str, 'married': str, 'urban': str}
train_y = train_data['voted']
train_x = train_data.drop(drop_list + ['voted'], axis=1)
test_x = test_data.drop(drop_list, axis=1)
train_x = train_x.astype(replace_dict)
test_x = test_x.astype(replace_dict)
train_x = pd.get_dummies(train_x)
test_x = pd.get_dummies(test_x)
train_y = 2 - train_y.to_numpy()
train_x = train_x.to_numpy()
test_x = test_x.to_numpy()

train_x[:, :20] = (train_x[:, :20] - 3.) / 2.
test_x[:, :20] = (test_x[:, :20] - 3.) / 2
train_x[:, 20] = (train_x[:, 20] - 5.) / 5.
test_x[:, 20] = (test_x[:, 20] - 5.) / 5.
train_x[:, 21:31] = (train_x[:, 21:31] - 3.5) / 3.5
test_x[:, 21:31] = (test_x[:, 21:31] - 3.5) / 3.5
train_y = torch.tensor(train_y, dtype=torch.float32)
train_x = torch.tensor(train_x, dtype=torch.float32)
test_x = torch.tensor(test_x, dtype=torch.float32)
train_len, test_len = len(train_x), len(test_x)

N_MODEL = 10
N_EPOCH = 40
BATCH_SIZE = 128
LOADER_PARAM = {
    'batch_size': BATCH_SIZE,
    'num_workers': 4,
    'pin_memory': True
}
prediction = np.zeros((11383, 1), dtype=np.float32)
predict = np.zeros((11383, 10), dtype=np.float32)

for no in range(N_MODEL):

    train_loader = DataLoader(TensorDataset(train_x, train_y),
                              shuffle=True, drop_last=True, **LOADER_PARAM)
    test_loader = DataLoader(TensorDataset(test_x, torch.zeros((test_len,), dtype=torch.float32)),
                             shuffle=False, drop_last=False, **LOADER_PARAM)
    model = nn.Sequential(
        nn.Dropout(0.05),
        nn.Linear(91, 104, bias=False),
        nn.BatchNorm1d(104),
        nn.LeakyReLU(0.05, inplace=True),
        nn.Dropout(0.2),
        nn.Linear(104, 80, bias=False),
        nn.BatchNorm1d(80),
        nn.LeakyReLU(0.05, inplace=True),
        nn.Linear(80, 64, bias=False),
        nn.BatchNorm1d(64),
        nn.LeakyReLU(0.05, inplace=True),
        nn.Dropout(0.5),
        nn.Linear(64, 32, bias=False),
        nn.BatchNorm1d(32),
        nn.ReLU(inplace=True),
        nn.Linear(32, 1)
    ).to(DEVICE)
    criterion = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1.20665], device=DEVICE))
    optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=4e-2)
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=N_EPOCH // 4, eta_min=1.2e-5)

    model.train()
    for epoch in tqdm(range(N_EPOCH), desc='{:02d}/{:02d}'.format(no + 1, N_MODEL)):
        for idx, (xx, yy) in enumerate(train_loader):
            optimizer.zero_grad()
            xx, yy = xx.to(DEVICE), yy.to(DEVICE)
            pred = model(xx).squeeze()
            loss = criterion(pred, yy)
            loss.backward()
            optimizer.step()
            scheduler.step(epoch + idx / len(train_loader))

    model.eval()
    with torch.no_grad():
        for idx, (xx, _) in enumerate(test_loader):
            xx = xx.to(DEVICE)
            pred = (2. - torch.sigmoid(model(xx).detach().to('cpu'))).numpy()
            prediction[BATCH_SIZE * idx:min(BATCH_SIZE * (idx + 1), len(prediction)), :] += pred[:, :] / N_MODEL
            predict[BATCH_SIZE * idx:min(BATCH_SIZE * (idx + 1), len(prediction)),no] = pred[:,0]

df = pd.read_csv('data/open data/sample_submission.csv')
df.iloc[:, 1:] = prediction
df.to_csv('data/open data/{}.csv'.format(datetime.now().strftime('%m%d-%H%M')), index=False)

10/10: 100%|██████████| 40/40 [01:39<00:00,  2.48s/it]


In [7]:
pred.shape

(128, 1)

In [130]:
pres = np.round(predict)
pres = np.mean(pres,axis=1)
pres.shape

(11383,)

In [140]:
train_x1 = torch.cat((train_x, test_x[np.where((pres ==1) | (pres==2))[0],:]), dim=0)
test_pre = 2 - pres[np.where((pres ==1) | (pres==2))]
test_pre = torch.tensor(test_pre, dtype=torch.float32)
train_y1 = torch.cat((train_y, test_pre))

In [115]:
train_predict = np.zeros((45440, 1), dtype=np.float32)
real = np.zeros((45440, 1), dtype=np.float32)
train_loader = DataLoader(TensorDataset(train_x, train_y),
                              shuffle=True, drop_last=True, **LOADER_PARAM)
test_loader = DataLoader(TensorDataset(test_x, torch.zeros((test_len,), dtype=torch.float32)),
                             shuffle=False, drop_last=False, **LOADER_PARAM)

with torch.no_grad():
    for idx, (xx, _) in enumerate(train_loader):
        xx = xx.to(DEVICE)
        pred = (2. - torch.sigmoid(model(xx).detach().to('cpu'))).numpy()
        train_predict[BATCH_SIZE * idx:min(BATCH_SIZE * (idx + 1), len(train_predict)), :] = pred[:, :]
        real[BATCH_SIZE * idx:min(BATCH_SIZE * (idx + 1), len(train_predict)),0] = 2 - _[:]
    print(BATCH_SIZE * (idx + 1))

45440


In [116]:
pd.crosstab(pd.Series(real.flat), pd.Series(np.round(train_predict).flat), rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,1.0,2.0,All
True,,,
1.0,17466,3125,20591
2.0,9543,15306,24849
All,27009,18431,45440


In [143]:
N_MODEL = 10
N_EPOCH = 35
BATCH_SIZE = 128
LOADER_PARAM = {
    'batch_size': BATCH_SIZE,
    'num_workers': 4,
    'pin_memory': True
}
prediction = np.zeros((11383, 1), dtype=np.float32)
# predict = np.zeros((11383, 10), dtype=np.float32)

for no in range(N_MODEL):

    train_loader = DataLoader(TensorDataset(train_x1, train_y1),
                              shuffle=True, drop_last=True, **LOADER_PARAM)
    test_loader = DataLoader(TensorDataset(test_x, torch.zeros((test_len,), dtype=torch.float32)),
                             shuffle=False, drop_last=False, **LOADER_PARAM)
    model = nn.Sequential(
        nn.Dropout(0.05),
        nn.Linear(91, 104, bias=False),
        nn.BatchNorm1d(104),
        nn.LeakyReLU(0.05, inplace=True),
        nn.Dropout(0.2),
        nn.Linear(104, 80, bias=False),
        nn.BatchNorm1d(80),
        nn.LeakyReLU(0.05, inplace=True),
        nn.Linear(80, 64, bias=False),
        nn.BatchNorm1d(64),
        nn.LeakyReLU(0.05, inplace=True),
        nn.Dropout(0.5),
        nn.Linear(64, 32, bias=False),
        nn.BatchNorm1d(32),
        nn.ReLU(inplace=True),
        nn.Linear(32, 1)
    ).to(DEVICE)
    criterion = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1.20665], device=DEVICE))
    optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=4e-2)
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=N_EPOCH // 4, eta_min=1.2e-5)

    model.train()
    for epoch in tqdm(range(N_EPOCH), desc='{:02d}/{:02d}'.format(no + 1, N_MODEL)):
        for idx, (xx, yy) in enumerate(train_loader):
            optimizer.zero_grad()
            xx, yy = xx.to(DEVICE), yy.to(DEVICE)
            pred = model(xx).squeeze()
            loss = criterion(pred, yy)
            loss.backward()
            optimizer.step()
            scheduler.step(epoch + idx / len(train_loader))

    model.eval()
    with torch.no_grad():
        for idx, (xx, _) in enumerate(test_loader):
            xx = xx.to(DEVICE)
            pred = (2. - torch.sigmoid(model(xx).detach().to('cpu'))).numpy()
            prediction[BATCH_SIZE * idx:min(BATCH_SIZE * (idx + 1), len(prediction)), :] += pred[:, :] / N_MODEL
            # predict[BATCH_SIZE * idx:min(BATCH_SIZE * (idx + 1), len(prediction)),no] = pred[:,0]

df = pd.read_csv('data/open data/sample_submission.csv')
df.iloc[:, 1:] = prediction
df.to_csv('data/open data/{}1.csv'.format(datetime.now().strftime('%m%d-%H%M')), index=False)

10/10: 100%|██████████| 30/30 [01:31<00:00,  3.07s/it]


In [141]:
torch.tensor([1.20665], device=DEVICE)

tensor([1.2067], device='cuda:0')